### You can also run the notebook in [COLAB](https://colab.research.google.com/github/deepmipt/DeepPavlov/blob/master/examples/trippy_tutorial.ipynb).

# TripPy Goal Oriented Bot in DeepPavlov

This tutorial describes how to build a Goal-Oriented Bot (Gobot) in DeepPavlov using the [TripPy architecture](https://arxiv.org/pdf/2005.02877.pdf).

This tutorial follows the same structure & uses the same data as the gobot_simple tutorial. We will only go over TripPy specific points here - so consult the gobot_simple notebook for general insights.

0. [Data preparation](#0.-Data-Preparation)
1. [Train bot](#1.-Train-bot)
2. [Interact with bot](#2.-Interact-with-bot)

In [1]:
!git clone -b rulebased_gobot_trippy https://github.com/DeepPavlov
%cd DeepPavlov
!pip install -r requirements.txt
!pip install transformers==2.9.1

Cloning into 'DeepPavlov'...
remote: Enumerating objects: 58503, done.
remote: Counting objects: 100% (1446/1446), done.
remote: Compressing objects: 100% (518/518), done.
remote: Total 58503 (delta 1089), reused 1225 (delta 915), pack-reused 57057
Receiving objects: 100% (58503/58503), 37.54 MiB | 25.87 MiB/s, done.
Resolving deltas: 100% (44932/44932), done.
/content/DeepPavlov
     |████████████████████████████████| 51kB 4.7MB/s 
     |████████████████████████████████| 2.1MB 7.2MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 2.9MB 29.0MB/s 
     |████████████████████████████████| 1.5MB 38.6MB/s 
     |████████████████████████████████| 20.1MB 1.4MB/s 
     |████████████████████████████████| 10.4MB 27.6MB/s 
     |████████████████████████████████| 512kB 33.9MB/s 
     |████████████████████████████████| 7.3MB 24.4MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 8.2MB 2.1MB/s 
     |██

     |████████████████████████████████| 645kB 5.1MB/s 
     |████████████████████████████████| 1.2MB 10.8MB/s 
     |████████████████████████████████| 5.6MB 22.4MB/s 


## 0. Data Preparation

The data preparation follows the exact same structure as the gobot_simple tutorial. Feel free to take a look there for specific insights.

In [2]:
from deeppavlov.dataset_readers.dstc2_reader import SimpleDSTC2DatasetReader


class AssistantDatasetReader(SimpleDSTC2DatasetReader):
    
    url = "http://files.deeppavlov.ai/datasets/tutor_assistant_data.tar.gz"
    
    @staticmethod
    def _data_fname(datatype):
        assert datatype in ('val', 'trn', 'tst'), "wrong datatype name"
        return f"assistant-{datatype}.json"

In [3]:
data = AssistantDatasetReader().read('assistant_data')

2021-07-13 17:44:39.271 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 283: [PosixPath('assistant_data/assistant-val.json'), PosixPath('assistant_data/assistant-tst.json')]]
2021-07-13 17:44:39.272 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 284: [downloading data from http://files.deeppavlov.ai/datasets/tutor_assistant_data.tar.gz to assistant_data]
2021-07-13 17:44:39.275 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/datasets/tutor_assistant_data.tar.gz to assistant_data/tutor_assistant_data.tar.gz
100%|██████████| 838/838 [00:00<00:00, 139kB/s]
2021-07-13 17:44:40.476 INFO in 'deeppavlov.core.data.utils'['utils'] at line 272: Extracting assistant_data/tutor_assistant_data.tar.gz archive into assistant_data
2021-07-13 17:44:40.483 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 304: [loading dialogs from assistant_data/assistant-trn.json]
2021-07-

In [4]:
from deeppavlov.dataset_iterators.dialog_iterator import DialogDatasetIterator

iterator = DialogDatasetIterator(data)

## 1. Train bot

The TripPy architecture without slots as implemented in DeepPavlov is illustrated in the below sketch:

&nbsp;
![trippy_architecture_simple.png](img/trippy_architecture_simple.png)
&nbsp;

User-uttarance + Dialogue History (if existing) --> Tokenize and otherwise prepare for TripPy --> Feed through the TripPy BERT model --> A Linear Head with a Softmax predicts the action --> NLG generates a sentence based on the predicted action


The large empty areas are fields that are only used when slot values are predicted. Refer to the trippy_extended tutorial to learn more about those.


Note that this architecture is very different from the original architecture you may find in their paper as the authors implement no action prediction. 



In [5]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json

# We use the TripPy DSTC2 minimal config
gobot_config = read_json(configs.go_bot.trippy_dstc2_minimal)

gobot_config['chainer']['pipe'][-1]['nlg_manager']['template_path'] = 'assistant_data/assistant-templates.txt'

In [6]:
gobot_config['chainer']['pipe'][-1]['nlg_manager']['template_path'] = 'assistant_data/assistant-templates.txt'
gobot_config['chainer']['pipe'][-1]['nlg_manager']['api_call_action'] = None

Specify train/valid/test data path and path to save the final bot model:

In [7]:
gobot_config['dataset_reader']['class_name'] = '__main__:AssistantDatasetReader'
gobot_config['metadata']['variables']['DATA_PATH'] = 'assistant_data'
gobot_config['metadata']['variables']['MODEL_PATH'] = 'assistant_bot'

In [67]:
# Set TripPy hyperparameters

gobot_config['train']['batch_size'] = 16 # set batch size
gobot_config['train']['max_batches'] = 64 # maximum number of training batches
gobot_config['train']['val_every_n_batches'] = 10 # evaluate on full 'valid' split every 30 epochs
gobot_config['train']['log_every_n_batches'] = 10 # evaluate on full 'train' split every 5 batches
gobot_config['train']['validation_patience'] = 10 # after no improvements on validation for 10 vals, stop training

gobot_config['chainer']['pipe'][-1]['slot_names'] = [] # no slot names for this dataset

gobot_config['chainer']['pipe'][-1]["optimizer_parameters"] = {"lr": 1e-4, "eps": 1e-6}

In [68]:
from deeppavlov import train_model

# Training should take ~70 seconds & reach 100% validation accuracy
train_model(gobot_config)

2021-07-13 17:52:17.608 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 304: [loading dialogs from /content/DeepPavlov/assistant_data/assistant-trn.json]
2021-07-13 17:52:17.612 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 304: [loading dialogs from /content/DeepPavlov/assistant_data/assistant-val.json]
2021-07-13 17:52:17.618 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 304: [loading dialogs from /content/DeepPavlov/assistant_data/assistant-tst.json]
2021-07-13 17:52:17.620 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 296: There are 24 samples in train split.
2021-07-13 17:52:17.622 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 297: There are 3 samples in valid split.
2021-07-13 17:52:17.624 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 298: There are 3 samples in test split.
/usr/local/lib/python3.7/dist-packages/torc

{"train": {"eval_examples_count": 8, "metrics": {"per_item_dialog_accuracy": 1.0}, "time_spent": "0:00:05", "epochs_done": 9, "batches_seen": 10, "train_examples_seen": 80, "total_loss": 14.516639709472656, "action_loss": 14.516639709472656}}
{"valid": {"eval_examples_count": 1, "metrics": {"per_item_dialog_accuracy": 1.0}, "time_spent": "0:00:05", "epochs_done": 9, "batches_seen": 10, "train_examples_seen": 80, "impatience": 0, "patience_limit": 10}}


2021-07-13 17:52:35.740 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 212: Did not improve on the per_item_dialog_accuracy of 1.0


{"train": {"eval_examples_count": 8, "metrics": {"per_item_dialog_accuracy": 1.0}, "time_spent": "0:00:15", "epochs_done": 19, "batches_seen": 20, "train_examples_seen": 160, "total_loss": 2.900639772415161, "action_loss": 2.900639772415161}}
{"valid": {"eval_examples_count": 1, "metrics": {"per_item_dialog_accuracy": 1.0}, "time_spent": "0:00:15", "epochs_done": 19, "batches_seen": 20, "train_examples_seen": 160, "impatience": 1, "patience_limit": 10}}


2021-07-13 17:52:40.512 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 212: Did not improve on the per_item_dialog_accuracy of 1.0


...

{"train": {"eval_examples_count": 8, "metrics": {"per_item_dialog_accuracy": 1.0}, "time_spent": "0:00:33", "epochs_done": 59, "batches_seen": 60, "train_examples_seen": 480, "total_loss": 0.0066082654520869255, "action_loss": 0.0066082654520869255}}
{"valid": {"eval_examples_count": 1, "metrics": {"per_item_dialog_accuracy": 1.0}, "time_spent": "0:00:33", "epochs_done": 59, "batches_seen": 60, "train_examples_seen": 480, "impatience": 5, "patience_limit": 10}}


2021-07-13 17:53:00.297 INFO in 'deeppavlov.models.go_bot.trippy'['trippy'] at line 152: Load path /content/DeepPavlov/assistant_bot/model is given.
2021-07-13 17:53:00.299 INFO in 'deeppavlov.models.go_bot.trippy'['trippy'] at line 159: Load path /content/DeepPavlov/assistant_bot/model.pth.tar exists.
2021-07-13 17:53:00.304 INFO in 'deeppavlov.models.go_bot.trippy'['trippy'] at line 160: Initializing `TripPy` from saved.
2021-07-13 17:53:00.306 INFO in 'deeppavlov.models.go_bot.trippy'['trippy'] at line 163: Loading weights from /content/DeepPavlov/assistant_bot/model.pth.tar.
2021-07-13 17:53:01.114 INFO in 'deeppavlov.core.models.torch_model'['torch_model'] at line 98: Model was successfully initialized! Model summary:
 BertForDST(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), ep

{"valid": {"eval_examples_count": 1, "metrics": {"per_item_dialog_accuracy": 1.0}, "time_spent": "0:00:01"}}
{"test": {"eval_examples_count": 1, "metrics": {"per_item_dialog_accuracy": 1.0}, "time_spent": "0:00:01"}}


2021-07-13 17:53:05.169 INFO in 'deeppavlov.models.go_bot.trippy'['trippy'] at line 152: Load path /content/DeepPavlov/assistant_bot/model is given.
2021-07-13 17:53:05.171 INFO in 'deeppavlov.models.go_bot.trippy'['trippy'] at line 159: Load path /content/DeepPavlov/assistant_bot/model.pth.tar exists.
2021-07-13 17:53:05.180 INFO in 'deeppavlov.models.go_bot.trippy'['trippy'] at line 160: Initializing `TripPy` from saved.
2021-07-13 17:53:05.183 INFO in 'deeppavlov.models.go_bot.trippy'['trippy'] at line 163: Loading weights from /content/DeepPavlov/assistant_bot/model.pth.tar.
2021-07-13 17:53:05.999 INFO in 'deeppavlov.core.models.torch_model'['torch_model'] at line 98: Model was successfully initialized! Model summary:
 BertForDST(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), ep

Chainer[<deeppavlov.models.go_bot.trippy.TripPy at 0x7fe5e46fa310>]

# 2. Interact with bot

In [69]:
from deeppavlov import build_model

bot = build_model(gobot_config)

/usr/local/lib/python3.7/dist-packages/torch/nn/init.py:388: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
2021-07-13 17:53:10.142 INFO in 'deeppavlov.models.go_bot.trippy'['trippy'] at line 152: Load path /content/DeepPavlov/assistant_bot/model is given.
2021-07-13 17:53:10.144 INFO in 'deeppavlov.models.go_bot.trippy'['trippy'] at line 159: Load path /content/DeepPavlov/assistant_bot/model.pth.tar exists.
2021-07-13 17:53:10.147 INFO in 'deeppavlov.models.go_bot.trippy'['trippy'] at line 160: Initializing `TripPy` from saved.
2021-07-13 17:53:10.150 INFO in 'deeppavlov.models.go_bot.trippy'['trippy'] at line 163: Loading weights from /content/DeepPavlov/assistant_bot/model.pth.tar.
2021-07-13 17:53:10.989 INFO in 'deeppavlov.core.models.torch_model'['torch_model'] at line 98: Model was successfully initialized! Model summary:
 BertForDST(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embe

##### Original Example from gobot_simple

In [70]:
bot(["good evening, bot"])

[['Hello, what is the weather today?']]

In [71]:
bot(["the weather is clooudy and gloooomy"])

[['Then you should cycle!']]

In [72]:
bot(["nice idea, thanks!"])

[['You are welcome! Bye!']]

In [73]:
bot.reset()

In [74]:
bot(["hi bot"])

[['Hello, what is the weather today?']]

In [75]:
bot(["looks ok, the sun is bright and yesterday's rain stopped already"])
# The bot isn't perfect!

[['Then you should cycle!']]

In [76]:
bot([ "i dont wanna"])

[["That's a pity! Next time maybe. Have a good day!"]]

##### New example

In [77]:
bot.reset()

In [78]:
bot(["hi bot"])

[['Hello, what is the weather today?']]

In [79]:
bot(["too much snow, dont want to go out"])

[['Then you should try hot chinese tea!']]

In [80]:
bot(["no i dont want tea"])

[["That's a pity! Next time maybe. Have a good day!"]]

In [81]:
bot.reset()

In [82]:
bot(["hi"])

[['Hello, what is the weather today?']]

In [83]:
bot(["it's sunny in california"])

[['Then you should cycle!']]

In [84]:
bot(["See you next time!"])

[['You are welcome! Bye!']]

In [85]:
bot.reset()

In [27]:
# For you to try